# 1) Setup

In [9]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import math
import scipy as sp
from scipy import stats
from copy import copy
import time

import pymongo
from pymongo import MongoClient
from datetime import datetime
from datetime import timedelta
print("done")

done


In [10]:
#import functions from python files

#1) data import & filtering
from get_data import *
from get_adjacent import *
from get_valid1 import *
from get_times import *
from get_congestion import *
from get_valid2 import *

from select_data import *
from select_data_edge import *

from underscore_prefix import *
from underscore_suffix import *

print("done")

done


# 2) AAF Year 4

In [11]:
#1) Import data from PyMongo
df = get_data(address = "aaf")
print(len(df))

#2) Remove data with invalid origin/target
adjacent = get_adjacent("aaf_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))
df.head(2)

23113
Time taken: 0.54353928565979 secs
23044


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node
0,583b30dfb63c4d6eb86ff88e,success,Station,ChargingPoint,Station_ChargingPoint,"Sunday, November 27 2016, at 20:15:43 hours","Sunday, November 27 2016, at 20:15:43 hours","Sunday, November 27 2016, at 20:15:24 hours","{'inserted_at': 2016-11-27 19:15:43.999000, 's...",0.624420,19.893930,ChargingPoint
1,583b31d7b63c4d6eb86ff88f,success,ChargingPoint,Station,ChargingPoint_Station,"Sunday, November 27 2016, at 20:19:51 hours","Sunday, November 27 2016, at 20:19:34 hours","Sunday, November 27 2016, at 20:19:27 hours","{'inserted_at': 2016-11-27 19:19:51.098000, 's...",17.028488,23.572407,Station


In [12]:
#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)
print("done")

Time taken (get_times): 18.731311321258545 secs
done


In [13]:
#4) Count congestion level
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))
df.head()

0 iterations in 0.0 seconds
5000 iterations in 22.29765772819519 seconds
10000 iterations in 47.71601486206055 seconds
15000 iterations in 72.10530304908752 seconds
20000 iterations in 94.61977577209473 seconds
Time taken (get_congestion): 108.47974109649658 seconds
Unimodal:  22581
Multimodal:  463


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
0,583b30dfb63c4d6eb86ff88e,success,Station,ChargingPoint,Station_ChargingPoint,2016-11-27 20:15:43,2016-11-27 20:15:43,2016-11-27 20:15:24,2016-11-27 19:15:43.999000,0.624420,19.893930,ChargingPoint,1
1,583b31d7b63c4d6eb86ff88f,success,ChargingPoint,Station,ChargingPoint_Station,2016-11-27 20:19:51,2016-11-27 20:19:34,2016-11-27 20:19:27,2016-11-27 19:19:51.098000,17.028488,23.572407,Station,1
2,583b322eb63c4d6eb86ff890,success,Station,ChargingPoint,Station_ChargingPoint,2016-11-27 20:21:18,2016-11-27 20:21:18,2016-11-27 20:20:59,2016-11-27 19:21:18.512000,0.345883,19.336452,ChargingPoint,1
3,583bde4ab63c4d6eb86ff8a6,success,ChargingPoint,Station,ChargingPoint_Station,2016-11-28 08:35:38,2016-11-28 08:35:24,2016-11-28 08:35:17,2016-11-28 07:35:38.749000,14.376565,21.574658,Station,1
4,583bde53b63c4d6eb86ff8a7,success,Station,WayPoint91,Station_WayPoint91,2016-11-28 08:35:47,2016-11-28 08:35:47,2016-11-28 08:35:38,2016-11-28 07:35:47.600000,0.132829,8.723706,WayPoint91,1


In [21]:
df[(df["target"] == df["final_node"]) & (df["edge_id"] == "Frisoer_WayPoint35") & (df["time_to_waypoint"] > 5)]

,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
4241,5857a93bb63c4d1165a845a6,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2016-12-19 10:32:43,2016-12-19 10:32:37,2016-12-19 10:32:30,2016-12-19 09:32:43.681000,5.728350,12.973170,WayPoint35,1
4259,5857ac75b63c4d1165a85251,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2016-12-19 10:46:29,2016-12-19 10:46:23,2016-12-19 10:46:16,2016-12-19 09:46:29.163000,5.899469,13.107954,WayPoint35,1
4277,5857afcab63c4d1165a85fa8,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2016-12-19 11:00:42,2016-12-19 11:00:37,2016-12-19 11:00:29,2016-12-19 10:00:42.487000,5.431196,12.672047,WayPoint35,1
4295,5857b35fb63c4d1165a8a13d,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2016-12-19 11:15:59,2016-12-19 11:15:54,2016-12-19 11:15:46,2016-12-19 10:15:59.406000,5.249186,13.077390,WayPoint35,1
4313,5857b758b63c4d1165a8cf3b,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2016-12-19 11:32:56,2016-12-19 11:32:51,2016-12-19 11:32:44,2016-12-19 10:32:56.808000,5.654114,12.622241,WayPoint35,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21502,58d24edab63c4d09837e4d57,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-22 11:15:54,2017-03-22 11:15:46,2017-03-22 11:15:38,2017-03-22 10:15:54.182000,7.369977,16.100099,WayPoint35,1
21520,58d25389b63c4d09837e6514,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-22 11:35:53,2017-03-22 11:35:48,2017-03-22 11:35:42,2017-03-22 10:35:53.900000,5.292322,11.323899,WayPoint35,1
21617,58d388bbb63c4d098380664f,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-23 09:35:07,2017-03-23 09:35:01,2017-03-23 09:34:55,2017-03-23 08:35:07.559000,6.349824,12.147270,WayPoint35,1
21631,58d38befb63c4d0983806bde,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-23 09:48:47,2017-03-23 09:48:41,2017-03-23 09:48:35,2017-03-23 08:48:47.316000,5.599571,12.171990,WayPoint35,1


In [20]:
df[(df["target"] != df["final_node"]) & (df["edge_id"] == "Frisoer_WayPoint35")& (df["n_robots"] == 1)]

,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
8722,5878c160b63c4d2c1089da66,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-01-13 13:00:33,2017-01-13 13:00:32,2017-01-13 12:59:54,2017-01-13 12:00:32.336000,0.235919,38.907433,Frisoer,1
12572,58986a38b63c4d2c10a9086c,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-02-06 13:21:15,2017-02-06 13:21:14,2017-02-06 13:20:35,2017-02-06 12:21:12.988000,0.201086,39.205955,WayPoint26,1
20932,58cfa8beb63c4d09837b7889,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-20 11:02:38,2017-03-20 11:02:36,2017-03-20 11:02:36,2017-03-20 10:02:38.950000,0.000000,2.694448,Frisoer,1


In [24]:
df[(df["edge_id"] == "Frisoer_WayPoint35")& (df["operation_time"] > 50)& (df["n_robots"] ==1)]

,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
7367,586fc02fb63c4d1165d4c5fb,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-01-06 17:05:04,2017-01-06 17:05:03,2017-01-06 17:03:59,2017-01-06 16:05:03.745000,0.146276,64.634806,WayPoint35,1
8433,58775e49b63c4d2c1087dbab,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-01-12 11:45:30,2017-01-12 11:45:29,2017-01-12 11:44:00,2017-01-12 10:45:29.408000,0.222076,89.765863,WayPoint35,1
8667,5878b304b63c4d2c10899be5,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-01-13 11:59:16,2017-01-13 11:59:16,2017-01-13 11:58:07,2017-01-13 10:59:16.076000,0.169582,69.103511,WayPoint35,1
14564,58a44bc9b63c4d17dc3c3fbe,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-02-15 13:38:36,2017-02-15 13:38:26,2017-02-15 13:37:03,2017-02-15 12:38:33.799000,10.169928,92.701398,WayPoint35,1
19036,58c2671fb63c4d09836c1fac,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-10 09:43:11,2017-03-10 09:41:12,2017-03-10 09:41:04,2017-03-10 08:43:11.040000,118.232733,126.385710,WayPoint35,1
22312,58da20dfb63c4d53cf5b5e43,success,Frisoer,WayPoint35,Frisoer_WayPoint35,2017-03-28 10:37:51,2017-03-28 10:37:51,2017-03-28 10:36:20,2017-03-28 08:37:51.553000,0.154479,91.165284,WayPoint35,1


In [27]:
df[(df["edge_id"] == "Frisoer_WayPoint35")]["operation_time"].mean
df[(df["edge_id"] == "Frisoer_WayPoint35")]["time_to_waypoint"].mean

<bound method Series.mean of 893      0.137048
908      0.155360
928      0.141602
946      0.153498
964      0.129828
           ...   
22695    0.193290
22709    0.145887
22722    0.165408
22736    0.110010
23006    0.096644
Name: time_to_waypoint, Length: 594, dtype: float64>

In [14]:
#5) remove final, unsuccessful, multimodal runs
df_aaf = get_valid2(df)
df_aaf.head()

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station,ChargingPoint,Station_ChargingPoint,0.624420,19.893930
1,ChargingPoint,Station,ChargingPoint_Station,17.028488,23.572407
2,Station,ChargingPoint,Station_ChargingPoint,0.345883,19.336452
3,ChargingPoint,Station,ChargingPoint_Station,14.376565,21.574658
4,Station,WayPoint91,Station_WayPoint91,0.132829,8.723706


In [15]:
#6) save to excel
df_aaf.to_excel("aaf_unimodal.xlsx")
print("done")

done


In [6]:
# retrieve data
df_aaf = pd.read_excel("aaf_unimodal.xlsx")
df_aaf = df_aaf[["origin", "target", "edge_id", "time_to_waypoint", "operation_time"]]
df_aaf.head()

,Unnamed: 0,origin,target,edge_id,time_to_waypoint,operation_time
0,0,Station,ChargingPoint,Station_ChargingPoint,0.624420,19.893930
1,1,ChargingPoint,Station,ChargingPoint_Station,17.028488,23.572407
2,2,Station,ChargingPoint,Station_ChargingPoint,0.345883,19.336452
3,3,ChargingPoint,Station,ChargingPoint_Station,14.376565,21.574658
4,4,Station,WayPoint91,Station_WayPoint91,0.132829,8.723706


In [8]:
df = df_aaf[df_aaf["edge_id"] == "Frisoer_WayPoint35"]
df.head(100)

,Unnamed: 0,origin,target,edge_id,time_to_waypoint,operation_time
891,893,Frisoer,WayPoint35,Frisoer_WayPoint35,0.137048,7.149220
906,908,Frisoer,WayPoint35,Frisoer_WayPoint35,0.155360,7.302483
926,928,Frisoer,WayPoint35,Frisoer_WayPoint35,0.141602,7.435694
944,946,Frisoer,WayPoint35,Frisoer_WayPoint35,0.153498,7.231636
962,964,Frisoer,WayPoint35,Frisoer_WayPoint35,0.129828,7.002821
...,...,...,...,...,...,...
5755,5852,Frisoer,WayPoint35,Frisoer_WayPoint35,5.256583,12.689250
5794,5891,Frisoer,WayPoint35,Frisoer_WayPoint35,5.763836,12.956265
5807,5904,Frisoer,WayPoint35,Frisoer_WayPoint35,0.176042,7.013156
5842,5939,Frisoer,WayPoint35,Frisoer_WayPoint35,0.109704,7.103612


# 3) TSC Betty (Year 4)

In [16]:
#1) Import data from PyMongo
df = get_data(address = "tsc_betty")
print(len(df))

#2) Remove data with invalid origin/target
adjacent = get_adjacent("tsc_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))
df.head(2)

3752
Time taken: 0.5555450916290283 secs
3587


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node
0,58525d7fb31be8487921ffe0,success,Station1,SmallKitchen,Station1_SmallKitchen,"Thursday, December 15 2016, at 09:08:15 hours","Thursday, December 15 2016, at 09:08:15 hours","Thursday, December 15 2016, at 09:08:06 hours","{'inserted_at': 2016-12-15 09:08:15.583000, 's...",0.047758,9.263602,SmallKitchen
1,58525d88b31be8487921ffe1,success,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,"Thursday, December 15 2016, at 09:08:24 hours","Thursday, December 15 2016, at 09:08:24 hours","Thursday, December 15 2016, at 09:08:15 hours","{'inserted_at': 2016-12-15 09:08:24.325000, 's...",0.088824,8.628814,HospActRec4


In [17]:
#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)
print("done")

Time taken (get_times): 1.2337124347686768 secs
done


In [21]:
#4) Count congestion level
df = df[df["target"] != "CorpTransWP1"].reset_index(drop = True)     #mistake in topological map file
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))
df.head()

0 iterations in 0.0009970664978027344 seconds
Time taken (filter_data2): 5.347533464431763 seconds


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
0,58525d7fb31be8487921ffe0,success,Station1,SmallKitchen,Station1_SmallKitchen,2016-12-15 09:08:15,2016-12-15 09:08:15,2016-12-15 09:08:06,2016-12-15 09:08:15.583000,0.047758,9.263602,SmallKitchen,1
1,58525d88b31be8487921ffe1,success,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,2016-12-15 09:08:24,2016-12-15 09:08:24,2016-12-15 09:08:15,2016-12-15 09:08:24.325000,0.088824,8.628814,HospActRec4,1
2,58525d8db31be8487921ffe2,success,HospActRec4,GlassCorridor3,HospActRec4_GlassCorridor3,2016-12-15 09:08:29,2016-12-15 09:08:29,2016-12-15 09:08:24,2016-12-15 09:08:29.742000,0.099887,5.299425,GlassCorridor3,1
3,58525e68b31be8487921ffe8,success,CorpTransWP1,CorpTransWP2,CorpTransWP1_CorpTransWP2,2016-12-15 09:12:08,2016-12-15 09:12:05,2016-12-15 09:11:04,2016-12-15 09:12:08.764000,3.428591,63.935443,CorpTransWP2,1
4,58525e75b31be8487921ffe9,success,CorpTransWP2,CorpTransWP4,CorpTransWP2_CorpTransWP4,2016-12-15 09:12:20,2016-12-15 09:12:20,2016-12-15 09:12:08,2016-12-15 09:12:21.007000,0.069340,12.130074,CorpTransWP4,1


In [23]:
#5) remove final, unsuccessful, multimodal runs
df_tsc_betty = get_valid2(df)
df_tsc_betty.head()

,origin,target,edge_id,time_to_waypoint,operation_time
0,Station1,SmallKitchen,Station1_SmallKitchen,0.047758,9.263602
1,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.088824,8.628814
2,HospActRec4,GlassCorridor3,HospActRec4_GlassCorridor3,0.099887,5.299425
3,CorpTransWP1,CorpTransWP2,CorpTransWP1_CorpTransWP2,3.428591,63.935443
4,CorpTransWP2,CorpTransWP4,CorpTransWP2_CorpTransWP4,0.069340,12.130074


In [24]:
#6) save to excel
df_tsc_betty.to_excel("tsc_betty_unimodal.xlsx")
print("done")

done


# 4) TSC Bob (Year 4)

In [25]:
#1) Import data from PyMongo
df = get_data(address = "tsc_bob")
print(len(df))

#2) Remove data with invalid origin/target
adjacent = get_adjacent("tsc_map.yaml")
df = get_valid1(df, adjacent)
print(len(df))
df.head(2)

9084
Time taken: 0.505157470703125 secs
9084


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node
0,58bfda853ee7491171a6d2bc,success,ChargingPoint1,Station1,ChargingPoint1_Station1,"Wednesday, March 08 2017, at 10:18:45 hours","Wednesday, March 08 2017, at 10:18:23 hours","Wednesday, March 08 2017, at 10:18:23 hours","{'inserted_at': 2017-03-08 10:18:45.613000, 's...",0.000000,22.567267,none
1,58bfda8b3ee7491171a6d2be,success,Station1,SmallKitchen,Station1_SmallKitchen,"Wednesday, March 08 2017, at 10:18:51 hours","Wednesday, March 08 2017, at 10:18:51 hours","Wednesday, March 08 2017, at 10:18:47 hours","{'inserted_at': 2017-03-08 10:18:51.581000, 's...",0.054096,4.340037,SmallKitchen


In [26]:
#3) Change format of datetimes in dataframe to datetime.datetime object
df = get_times(df)
print("done")

Time taken (get_times): 4.1710004806518555 secs
done


In [27]:
#4) Count congestion level
df = df[df["target"] != "CorpTransWP1"].reset_index(drop = True)     #mistake in topological map file
df = get_congestion(df, adjacent)
print("Unimodal: ", len(df[df["n_robots"] == 1]))
print("Multimodal: ", len(df[df["n_robots"] != 1]))
df.head()

0 iterations in 0.0 seconds
5000 iterations in 12.678728818893433 seconds
Time taken (filter_data2): 22.738630533218384 seconds
8950
26


,_id,status,origin,target,edge_id,date_finished,date_at_node,date_started,_meta,time_to_waypoint,operation_time,final_node,n_robots
0,58bfda853ee7491171a6d2bc,success,ChargingPoint1,Station1,ChargingPoint1_Station1,2017-03-08 10:18:45,2017-03-08 10:18:23,2017-03-08 10:18:23,2017-03-08 10:18:45.613000,0.000000,22.567267,none,1
1,58bfda8b3ee7491171a6d2be,success,Station1,SmallKitchen,Station1_SmallKitchen,2017-03-08 10:18:51,2017-03-08 10:18:51,2017-03-08 10:18:47,2017-03-08 10:18:51.581000,0.054096,4.340037,SmallKitchen,1
2,58bfda913ee7491171a6d2bf,success,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,2017-03-08 10:18:57,2017-03-08 10:18:57,2017-03-08 10:18:51,2017-03-08 10:18:57.602000,0.077031,5.906566,HospActRec4,1
3,58bfda943ee7491171a6d2c0,success,HospActRec4,GlassCorridor3,HospActRec4_GlassCorridor3,2017-03-08 10:19:00,2017-03-08 10:19:00,2017-03-08 10:18:57,2017-03-08 10:19:00.926000,0.099578,3.207889,GlassCorridor3,1
4,58bfdae43ee7491171a6d2e1,success,CorpTransWP1,CorpTransWP2,CorpTransWP1_CorpTransWP2,2017-03-08 10:20:20,2017-03-08 10:20:17,2017-03-08 10:20:11,2017-03-08 10:20:20.054000,2.725526,8.978751,CorpTransWP2,1


In [28]:
#5) remove final, unsuccessful, multimodal runs
df_tsc_bob = get_valid2(df)
df_tsc_bob.head()

,origin,target,edge_id,time_to_waypoint,operation_time
1,Station1,SmallKitchen,Station1_SmallKitchen,0.054096,4.340037
2,SmallKitchen,HospActRec4,SmallKitchen_HospActRec4,0.077031,5.906566
3,HospActRec4,GlassCorridor3,HospActRec4_GlassCorridor3,0.099578,3.207889
4,CorpTransWP1,CorpTransWP2,CorpTransWP1_CorpTransWP2,2.725526,8.978751
5,CorpTransWP2,CorpTransWP4,CorpTransWP2_CorpTransWP4,0.054286,9.809353


In [29]:
#6) save to excel
df_tsc_bob.to_excel("tsc_bob_unimodal.xlsx")
print("done")

done
